In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Activation, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, InputLayer, Flatten, BatchNormalization, Reshape, Lambda

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras.utils import multi_gpu_model
from IPython.display import Image 

import keras_metrics
import pandas
import ast
import numpy as np
import matplotlib.patches as patches 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageDraw

Using TensorFlow backend.


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import keras.backend as K
import numpy as np
import random

In [4]:
BATCH_SIZE = 32
IMG_SIZE = 224

arr1 = np.random.randint(245*3, size=(3,7,7,5))
arr2 = np.random.randint(245*3, size=(3,7,7,5))

arr3 = np.random.randint(245*3, size=(245*3))


#y_pred = np.reshape(y_pred, (BATCH_SIZE, 7,7,5))
batch_size = tensorflow.shape(arr3).numpy()[0]/245
print(tensorflow.shape(arr3).numpy())
print(batch_size)
#y_pred = tensorflow.reshape(y_pred, [batch_size, 7, 7, 5])
#y_true = np.reshape(y_true, (BATCH_SIZE, 7,7,5))
#y_true = tensorflow.reshape(y_true, [batch_size, 7, 7, 5])


b_p_pred = arr2[..., 0]
b_x_pred = arr2[..., 1]
b_y_pred = arr2[..., 2]
b_w_pred = arr2[..., 3]
b_h_pred = arr2[..., 4]


b_p = arr1[..., 0]
b_x = arr1[..., 1]
b_y = arr1[..., 2]
b_w = arr1[..., 3]
b_h = arr1[..., 4]

#print(b_xy_pred.get_shape(),b_xy.get_shape())
#print(b_wh_pred.get_shape(),b_wh.get_shape())
#indicator_coord = K.expand_dims(y_true[..., 3], axis=-1) * 1.0
loss_p = K.sum(K.square(b_p - b_p_pred), axis=-1)
loss_xy = K.sum(b_p * (K.square(b_x - b_x_pred) + K.square(b_y - b_y_pred)), axis=-1)# * indicator_coord)#, axis=[1,2,3,4])
loss_wh = K.sum(
    b_p * 
    (
        (K.square(K.sqrt(K.cast(b_w, dtype="float32")) - K.sqrt(K.cast(b_w_pred, dtype="float32")))) + 
        (K.square(K.sqrt(K.cast(b_h, dtype="float32")) - K.sqrt(K.cast(b_h_pred, dtype="float32"))))
    ), axis=-1)# * indicator_coord)#, axis=[1,2,3,4])

K.print_tensor(K.sqrt( K.cast(b_w, dtype="float32") ) ,message='b_w: = ')

K.print_tensor(loss_p, message="loss_p: ")
K.print_tensor(loss_xy, message="loss_xy: ")
K.print_tensor(loss_wh, message="loss_wh: ")

#print(K.cast(loss_p, dtype="float32"))
#print(K.cast(loss_xy, dtype="float32"))
#print(loss_wh)
K.print_tensor(( K.cast(loss_p, dtype="float32") + loss_wh +  K.cast(loss_xy, dtype="float32") )/3, message="loss: ")
print (( K.cast(loss_p, dtype="float32") + loss_wh +  K.cast(loss_xy, dtype="float32") )/3)

In [5]:
def yolf_loss(y_true, y_pred):
    #true e pred sono [32][245]
    
    #tensorflow.print("tensors:", y_pred)
    #tensorflow.print("batch_size:", tensorflow.shape(y_pred)[0])
    
    y_pred = tensorflow.reshape(y_pred, [tensorflow.shape(y_pred)[0], 7, 7, 5])
    y_true = tensorflow.reshape(y_true, [tensorflow.shape(y_pred)[0], 7, 7, 5])


    b_p_pred = y_pred[..., 0]
    b_x_pred = y_pred[..., 1]
    b_y_pred = y_pred[..., 2]
    b_w_pred = y_pred[..., 3]
    b_h_pred = y_pred[..., 4]


    b_p = y_true[..., 0]
    b_x = y_true[..., 1]
    b_y = y_true[..., 2]
    b_w = y_true[..., 3]
    b_h = y_true[..., 4]

    #print(b_xy_pred.get_shape(),b_xy.get_shape())
    #print(b_wh_pred.get_shape(),b_wh.get_shape())
    #indicator_coord = K.expand_dims(y_true[..., 3], axis=-1) * 1.0
    loss_p = K.sum(K.square(b_p - b_p_pred), axis=-1)
    loss_xy = 5*K.sum(b_p * (K.square(b_x - b_x_pred) + K.square(b_y - b_y_pred)), axis=-1)# * indicator_coord)#, axis=[1,2,3,4])
    
    b_w = K.pow(b_w, 0.5)
    b_h = K.pow(b_h, 0.5)
    b_w_pred = K.pow(b_w_pred, 0.5)
    b_h_pred = K.pow(b_h_pred, 0.5)
    
    loss_wh = 5*K.sum(
        b_p * 
        (
            (K.square(b_w - b_w_pred)) + 
            (K.square(b_h - b_h_pred))
        ), axis=-1)# * indicator_coord)#, axis=[1,2,3,4])

    #tensorflow.print("loss_p:", loss_p)
    #tensorflow.print("loss_xy:", loss_xy)
    #tensorflow.print("loss_wh:", loss_wh)

    #print(K.cast(loss_p, dtype="float32"))
    #print(K.cast(loss_xy, dtype="float32"))
    #print(loss_wh)
    #tensorflow.print("Loss:", ( K.cast(loss_p, dtype="float32") + loss_wh +  K.cast(loss_xy, dtype="float32") )/3)
    return ( K.cast(loss_p, dtype="float32") + loss_wh +  K.cast(loss_xy, dtype="float32") )/3

In [6]:
train_set = ".\\dataset\\NUOVO\\train\\"
validation_set = ".\\dataset\\NUOVO\\val\\"
test_set = ".\\dataset\\NUOVO\\test\\"

In [7]:
train_df = pandas.read_csv(".\\dataset\\NUOVO\\train\\output_segmentation.csv")
valid_df = pandas.read_csv(".\\dataset\\NUOVO\\val\\output_segmentation.csv")
test_df = pandas.read_csv(".\\dataset\\NUOVO\\test\\output_segmentation.csv")

train_df['label'] = train_df['label'].apply(lambda x: ast.literal_eval(x))
valid_df['label'] = valid_df['label'].apply(lambda x: ast.literal_eval(x))
test_df['label'] = test_df['label'].apply(lambda x: ast.literal_eval(x))

IMG_SIZE = 224
BLOCKS_NUM = 19
BLOCK_SIZE = IMG_SIZE / BLOCKS_NUM

path_train_image = ".\\dataset\\NUOVO\\train"
count = 0 
for index,row in train_df.iterrows(): 
    im = Image.open(".\\dataset\\NUOVO\\train\\" + row["image_id"], 'r')
    draw = ImageDraw.Draw(im)
    for i in range(BLOCKS_NUM):
        for j in range(BLOCKS_NUM):

            #print(ast.literal_eval(row["label"])[(i*19)+j])
            
            if(row["label"][(i*19)+j][0] == 1):
                draw.rectangle(((i * BLOCK_SIZE, j * BLOCK_SIZE), (i * BLOCK_SIZE + BLOCK_SIZE, j * BLOCK_SIZE + BLOCK_SIZE)), outline="Green")
            else:
                draw.rectangle(((i * BLOCK_SIZE, j * BLOCK_SIZE), (i * BLOCK_SIZE + BLOCK_SIZE, j * BLOCK_SIZE + BLOCK_SIZE)), outline="Black")
    count += 1
    im.show()
    if(count == 1):
        break
    

train_df['label'] = train_df['label'].apply(lambda x: np.asarray(x).flatten())
valid_df['label'] = valid_df['label'].apply(lambda x: np.asarray(x).flatten())
test_df['label'] = test_df['label'].apply(lambda x: np.asarray(x).flatten())

for el in range(valid_df.label[0].shape[0]):
    valid_df[str(el)] = 0

valid_df.head(3)

len(valid_df.label[0])

count = 0
for index, row in valid_df.iterrows():
    count = count +1
    if count%1000==0:
        print("Done", count, "rows")
    for el in range(valid_df.label[0].shape[0]):
        valid_df[str(el)][index] = valid_df['label'][index][el]

for el in range(valid_df.label[0].shape[0]):
    print(valid_df[str(el)][0])

valid_df.head(3)

In [8]:
valid_df.columns[1:]

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '235', '236', '237', '238', '239', '240', '241', '242', '243', '244'],
      dtype='object', length=245)

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=train_set,
        x_col="image_id",
        y_col=train_df.columns[1:],
        class_mode="raw",
        shuffle=True,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)

valid_generator = train_datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory=validation_set,
        x_col="image_id",
        y_col=valid_df.columns[1:],
        class_mode="raw",
        shuffle=True,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)

Found 164105 validated image filenames.
Found 18234 validated image filenames.


train_generator.reset()

for el in train_generator:
    images = el[0]
    label = el[1]
    for l in label:
        print(l)

In [10]:
step_size_train = train_generator.samples/train_generator.batch_size
step_size_valid = valid_generator.samples/valid_generator.batch_size

true_boxes  = Input(shape=(1, 1, 1, 1, 1))
#input_image = Input(shape=(224, 224, 3))
input_image = InputLayer(input_shape=(224,224,3))

model = Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu")(input_image)
model = BatchNormalization()(model)
model = MaxPooling2D(pool_size=(2,2), strides=2)(model)

model = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(model)
model = BatchNormalization()(model)
model = MaxPooling2D(pool_size=(2,2), strides=2)(model)

model = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(model)
model = Conv2D(filters=64, kernel_size=(1,1), activation="relu")(model)
model = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(model)
model = BatchNormalization()(model)
model = MaxPooling2D(pool_size=(2,2), strides=2)(model)

model = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(model)
model = Conv2D(filters=128, kernel_size=(1,1), activation="relu")(model)
model = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(model)
model = BatchNormalization()(model)
model = MaxPooling2D(pool_size=(2,2), strides=2)(model)

model = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(model)
model = Conv2D(filters=256, kernel_size=(1,1), activation="relu")(model)
model = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(model)
model = Conv2D(filters=256, kernel_size=(1,1), activation="relu")(model)
model = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(model)
model = BatchNormalization()(model)
model = MaxPooling2D(pool_size=(2,2), strides=2)(model)
          
model = Conv2D(filters=1024, kernel_size=(3,3), padding="same", activation="relu")(model)
model = Conv2D(filters=512, kernel_size=(1,1), activation="relu")(model)
model = Conv2D(filters=1024, kernel_size=(3,3), padding="same", activation="relu")(model)
model = Conv2D(filters=512, kernel_size=(1,1), activation="relu")(model)
model = Conv2D(filters=1024, kernel_size=(3,3), padding="same", activation="relu")(model)

model = Conv2D(filters=5, kernel_size=(1,1), activation="relu")(model)

model = Reshape((7, 7, 5))(model)

#output = Lambda(lambda args: args[0])([model, true_boxes])

model = Sequential(input_image, output)


#model = Conv2D(filters=4, kernel_size=(1,1)))
#model = GlobalAveragePooling2D())
#model = BatchNormalization())

In [11]:
model = Sequential()
model.add(InputLayer(input_shape=(224,224,3)))
model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=64, kernel_size=(1,1), activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(1,1), activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(1,1), activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(1,1), activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
          
model.add(Conv2D(filters=1024, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(1,1), activation="relu"))
model.add(Conv2D(filters=1024, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(1,1), activation="relu"))
model.add(Conv2D(filters=1024, kernel_size=(3,3), padding="same", activation="relu"))

model.add(Conv2D(filters=5, kernel_size=(1,1), activation="relu"))

model.add(Flatten())
model.add(Dense(4096, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(245, activation="relu"))
model.add(Dropout(0.5))

#output = Lambda(lambda args: args[0])([model, true_boxes])

#model = Sequential(input_image, output)

#model.add(Conv2D(filters=4, kernel_size=(1,1)))
#model.add(GlobalAveragePooling2D())
#model.add(BatchNormalization())

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 224, 224, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 112, 112, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 128)       7

model = multi_gpu_model(model, gpus=2) #parallelize model

In [13]:
model.compile(optimizer=Adam(lr = 1e-3), loss=yolf_loss, metrics=[yolf_loss])

In [14]:
model.evaluate_generator(valid_generator, steps=step_size_valid, verbose = 1)

570/569 [==============================] - 35s 61ms/step - loss: 115544.3372 - yolf_loss: 115540.1719


[115582.29111824065, 115540.17]

In [15]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=50, verbose=1, mode='min')
mcp_save = ModelCheckpoint('yolf.h5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, mode='min')

history = model.fit_generator(generator=train_generator, epochs=3, steps_per_epoch=step_size_train, validation_data=valid_generator, validation_steps=step_size_valid, verbose=1, callbacks=[mcp_save, earlyStopping, reduce_lr_loss])

Epoch 1/3
 584/5128 [==>...........................] - ETA: 11:57 - loss: 115778.7685 - yolf_loss: 115778.8281

KeyboardInterrupt: 

In [ ]:
print(history.history)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training accuracy')
plt.plot(val_acc, label='Validation accuracy')
plt.legend(loc='lower right')
plt.ylabel('accuracy')
plt.ylim([min(plt.ylim()),500])
plt.title('Training and Validation accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,20000])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


# In[ ]:


print(history.history)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)


test_generator = test_datagen.flow_from_dataframe(dataframe=test_df,
                                    directory=test_set,
                                    x_col="image_id",
                                    y_col=["x_1","y_1","width","height"],
                                    class_mode="raw",
                                    shuffle=True,
                                    target_size=(224, 224),
                                    batch_size=BATCH_SIZE)

STEP_SIZE_TEST = test_generator.n / test_generator.batch_size 


#CHANGE PARALLEL MODEL
pred=model.predict_generator(test_generator,  steps=STEP_SIZE_TEST,  verbose=1)


In [ ]:
print(pred[1])

In [ ]:
# arrange output of pred
min_x, min_y = 100000,100000
max_x, max_y = -1,-1

BLOCK_SIZE = 32
for predict in pred:
    #np arrange qualcosa con 49 x 5
    for el in arranged:
        if el[0] == 1:
            temp_width, temp_height = el[3]*BLOCK_SIZE, el[4]*BLOCK_SIZE,
            temp_lt_x, temp_lt_y = el[1]-temp_width/2, el[2]-temp_heigth/2
            temp_rb_x, temp_rb_y = el[1]-temp_width/2, el[2]-temp_heigth/2
            if temp_lt_x<=min_x and temp_lt_y<=min_y:
                min_x, min_y= temp_lt_x,temp_lt_y
            else if temp_rb_x>=max_x and temp_rb_y>=max_y:
                max_x, max_y= temp_lt_x,temp_lt_y

print(min_x, min_y,max_x, max_y)

In [ ]:
from PIL import  Image, ImageDraw

count = 0

for gen_image in test_generator:
    img = Image.fromarray(np.uint8((gen_image[0][0])*255))
    img1 = ImageDraw.Draw(img)
    x1,y1 = pred[count][0],pred[count][1]
    x4,y4= pred[count][0] + pred[count][2], pred[count][1] + pred[count][3]
    img1.rectangle([(x1,y1),(x4,y4)], outline ="red") 
    #img.show() 
    img.save("/home/lorenzo.stacchio/ML/Machine_learning_project/dataset/NUOVO/results/output"+ str(count) + ".jpg")
    #print(gen_image[0][0].shape,pred[count])
    count = count + 1
    if count == 1000:
        break